In [1]:
import pandas as pd

# Load Train data
train_df = pd.read_csv('../data/TrainingDataPD25/TrainActivities.csv')
train_df.head()

,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject
0,1130251,2806,1 (FACING camera) Sit and stand,2024/09/02 6:16,2024/09/02 6:16,2024/09/02 6:16,U22
1,1130254,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024/09/02 6:17,2024/09/02 6:17,2024/09/02 6:17,U22
2,1130257,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024/09/02 6:18,2024/09/02 6:18,2024/09/02 6:18,U22
3,1130261,2806,1 (FACING camera) Sit and stand,2024/09/02 6:20,2024/09/02 6:20,2024/09/02 6:20,U22
4,1130292,2806,1 (FACING camera) Sit and stand,2024/09/02 6:42,2024/09/02 6:42,2024/09/02 6:42,U2


In [ ]:
# convert Started, Finished and Updated columns to datetime format
train_df['Started'] = pd.to_datetime(train_df['Started'])
train_df['Finished'] = pd.to_datetime(train_df['Finished'])
train_df['Updated'] = pd.to_datetime(train_df['Updated'])

In [3]:
# convert the timezone to JST
train_df['Started'] = train_df['Started'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')
train_df['Finished'] = train_df['Finished'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')
train_df['Updated'] = train_df['Updated'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')

In [ ]:
# cleaning train_df
train_df = train_df.dropna()

In [29]:
# combine all dataframes of each user 
import os
import glob

base_dir = '/Users/tptn/MachineLearning/tremor_challenge2/data/TrainingDataPD25/users_timeXYZ/users'

headers = ['id', 'datetime', 'x', 'y', 'z']
df_list = []
for user in os.listdir(base_dir):
    path = os.path.join(base_dir, user)

    if os.path.isdir(path):
        files = glob.glob(os.path.join(path, '*.csv'))

        for f in files:
            df = pd.read_csv(f, header=None, names=headers)
            df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M:%S.%f%z')
            if df['datetime'].dt.tz != 'UTC+09:00': # convert timezone if it not in JST
                df['datetime'] = df['datetime'].dt.tz_convert('Asia/Tokyo')
            df_list.append(df)
            
users_data = pd.concat(df_list, ignore_index=True)

In [39]:
users_data.shape
users_data.dtypes
# there are no null values in the dataframes

id                               int64
datetime    datetime64[ns, Asia/Tokyo]
x                              float64
y                              float64
z                              float64
dtype: object

In [ ]:
# train_df['Activity Type'].value_counts().index
# train_df['Activity Type'].unique()

array(['1 (FACING camera) Sit and stand',
       '2 (FACING camera) both hands SHAKING (sitting position)',
       '3 Stand up from chair - both hands with SHAKING',
       '4 (Sideway) Sit & stand',
       '5 (Sideway) both hands SHAKING (sitting)',
       '6 (Sideway) STAND up with - both hands SHAKING',
       '7 Cool down - sitting/relax', '8 Walk (LEFT --> Right --> Left)',
       '9 Walk & STOP/frozen, full body shaking, rotate then return back',
       '10 Slow walk (SHAKING hands/body, tiny step, head forward)'],
      dtype=object)

In [ ]:
ACT_LABELS = list(train_df['Activity Type'].unique())
# print(ACT_LABELS)
USERS = list(users_data['id'].unique())
# print(USERS)

In [49]:
# Split accelerometer data per 1 sample (=Activity at one time)
"""
Make Segment.
A segment is a group of continuous sensor data for which feature calculation (statistic calculation) is performed in human activity recognition using sensor data.
"""
seg_label_list = [] # segment's (1 sample's) label list (label mean "activity_type_id", "user_id" etc.).
seg_list = [] # segment's (1 sample's) accelerometer data list.

for i in list(train_df.index):
    started_at = train_df.loc[i, 'Started']
    finished_at = train_df.loc[i, 'Finished']
    seg = users_data[(users_data["datetime"] >=started_at) & (users_data["datetime"] <= finished_at)]
    seg_label = train_df.loc[i, "Activity Type"]
    if (len(seg)!=0):
        seg_list.append(seg)
        seg_label_list.append(seg_label)

seg_list[0].head()

,id,datetime,x,y,z
1343454,2807,2024-09-06 06:25:28.643000+09:00,2.452,-0.986,9.768
1343455,2807,2024-09-06 06:25:28.802000+09:00,2.433,-0.991,9.773
1343456,2807,2024-09-06 06:25:28.965000+09:00,2.449,-0.989,9.764
1343457,2807,2024-09-06 06:25:29.122000+09:00,2.428,-0.986,9.756
1343458,2807,2024-09-06 06:25:29.282000+09:00,2.425,-0.989,9.771
